##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Face Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect face landmarks from images.

## Preparation

Let's start with installing MediaPipe.

In [2]:
!pip install mediapipe --quiet 

Then download the off-the-shelf model bundle(s). Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker#models) for more information about these model bundles.

## Visualization utilities

In [1]:
#@markdown We implemented some functions to visualize the face landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt

2024-05-19 22:38:17.084402: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Running inference and visualizing the results

Here are the steps to run face landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python) to learn more about configuration options that this task supports.


In [1]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("data/img/oykueren/1.jpg")

# STEP 4: Detect face landmarks from the input image.
detection_result = detector.detect(image)

2024-05-20 13:57:46.814348: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1716202675.373495  498807 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-22.5.10), renderer: Intel(R) Iris(TM) Plus Graphics 645
W0000 00:00:1716202675.406639  498807 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1716202675.467200  518647 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716202675.498820  518643 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors

In [3]:
detection_result.face_landmarks

[[NormalizedLandmark(x=0.5699548721313477, y=0.6031114459037781, z=-0.02694292925298214, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5704045295715332, y=0.5592126250267029, z=-0.05079740658402443, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5702368021011353, y=0.5725946426391602, z=-0.026761915534734726, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5625861287117004, y=0.5150223970413208, z=-0.037636302411556244, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5703500509262085, y=0.5448697209358215, z=-0.053895555436611176, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5702881813049316, y=0.5272477269172668, z=-0.05002643167972565, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5703453421592712, y=0.4850139617919922, z=-0.024166543036699295, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.5069571137428284, y=0.4760036766529083, z=0.014817332848906517, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.56994891166

In [17]:
for i in detection_result.face_landmarks[0]:
    np.ndarray([i.x, i.y, i.z], dtype=np.float32)

TypeError: 'float' object cannot be interpreted as an integer

In [6]:
import plotly.graph_objects as go

# Assuming points_3d is your array of 3D points

# Create a scatter plot of the 3D points
fig = go.Figure(data=[go.Scatter3d(
    x=[int(landmark.x * 95) for landmark in detection_result.face_landmarks[0]],
    y=[int(landmark.y * 1358) for landmark in detection_result.face_landmarks[0]],
    z=[landmark.z for landmark in detection_result.face_landmarks[0]],
    mode='markers',
    marker=dict(
        size=2,
        color=[landmark.z for landmark in detection_result.face_landmarks[0]],  # color points by Z coordinate
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8
    )
)])

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    width=800,
    height=800,
    margin=dict(l=10, r=10, b=10, t=10)
)

# Show the plot
fig.show()